In [16]:
import faiss
import time
import numpy as np

from FlagEmbedding import BGEM3FlagModel
from backend.docstore import DocumentStore
from backend.retriever import FaissRetriever


### 검증용 모의 데이터

In [17]:
conversations = [
    ("user", "안녕하세요!"),
    ("assistant", "안녕하세요! 반갑습니다. 무엇을 도와드릴까요?"),
    ("user", "요즘 삶이 너무 무료하고 심심해요."),
    ("assistant", "그렇다면 새로운 취미를 찾아보는 건 어떨까요? 예를 들어 사이클이나 런닝같은 활동적인 것들은 어떤가요?"),
    ("user", "오늘 날씨가 어때요?"),
    ("assistant", "죄송하지만 저는 실시간 날씨 정보를 제공할 수 없습니다. 대신 일반적인 날씨 관련 조언을 드릴 수 있어요."),
    ("user", "그럼 여름철 더위 대비 팁 좀 알려주세요."),
    ("assistant", "여름철 더위 대비 팁을 알려드리겠습니다. 1. 충분한 수분 섭취 2. 헐렁한 옷 입기 3. 그늘에서 쉬기 4. 선크림 바르기 5. 에어컨 적절히 사용하기"),
    ("user", "고마워요. 이제 다른 주제로 넘어갈게요. 취미 추천해주세요."),
    ("assistant", "취미 추천 드리겠습니다. 1. 독서 2. 요리 3. 그림 그리기 4. 운동 (조깅, 요가 등) 5. 악기 연주 6. 정원 가꾸기 7. 사진 촬영 8. 외국어 공부. 이 중에서 관심 있는 것이 있나요?"),
    ("user", "음악에 관심이 있어요. 피아노를 배우고 싶은데 어떻게 시작하면 좋을까요?"),
    ("assistant", "피아노를 배우고 싶으시군요! 좋은 선택입니다. 시작하는 방법을 알려드리겠습니다. 1. 기초 악보 읽기 연습 2. 간단한 디지털 피아노나 키보드 구매 3. 온라인 무료 레슨 영상 활용 4. 근처 음악학원 등록 고려 5. 매일 조금씩이라도 연습하기. 천천히 시작해서 꾸준히 하는 것이 중요해요!"),
    ("user", "좋은 조언 고마워요. 이제 좀 피곤한데 재미있는 이야기 해줄래요?"),
    ("assistant", "물론이죠! 재미있는 이야기를 해드리겠습니다. 옛날 옛적에 한 농부가 있었어요. 이 농부는 매일 아침 닭장에 가서 황금알을 낳는 거위를 확인했답니다. 어느 날, 농부는 욕심이 생겨 '거위 배 안에 황금 덩어리가 있을 거야!'라고 생각했어요. 그래서 거위를 잡아 배를 갈랐죠. 하지만 안타깝게도 배 안에는 아무것도 없었고, 황금알을 낳는 거위는 죽고 말았답니다. 이 이야기의 교훈은 '욕심을 부리다 보면 가진 것마저 잃을 수 있다'는 거예요. 재미있게 들으셨나요?"),
    ("user", "네, 재미있었어요. 이제 자러 갈게요. 잘자요!"),
    ("assistant", "네, 잘 들어주셔서 감사합니다. 좋은 꿈 꾸세요! 편안한 밤 되세요. 내일도 좋은 하루 되길 바랄게요. 안녕히 주무세요!")
]

In [18]:
embedding_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
index = faiss.IndexFlatL2(1024) # bge-m3: 1024 dim
docstore = DocumentStore('seolpyo')

retriever = FaissIndex(embedding_model, index, docstore, normalize_L2=True)

Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]
INFO:FlagEmbedding.BGE_M3.modeling:loading existing colbert_linear and sparse_linear---------


### 대화 벡터들을 인덱스에 저장

In [19]:
for role, text in conversations:
    docstore.add(role, text)
    if role == "assistant":
        retriever.add_to_index(docstore.history[-1])
    time.sleep(np.random.uniform(1, 5))

INFO:docstore:Added message: Message(message={'role': 'user', 'content': '안녕하세요!'}, metadatas={'_id': '6980dade', 'timestamp': 1722791794.878166, 'session_id': 'f56f808b'})
INFO:docstore:Added message: Message(message={'role': 'assistant', 'content': '안녕하세요! 반갑습니다. 무엇을 도와드릴까요?'}, metadatas={'_id': '89e90c14', 'timestamp': 1722791799.399368, 'session_id': 'f56f808b'})
INFO:docstore:Updated history: {'text': '### 명령어: 안녕하세요!\n### 응답: 안녕하세요! 반갑습니다. 무엇을 도와드릴까요?', 'relative_time': 5.845994710922241}
INFO:retriever:Added 1 history vector to the index
INFO:retriever:Added 5.845994710922241sec to relative timestamps
INFO:docstore:Added message: Message(message={'role': 'user', 'content': '요즘 삶이 너무 무료하고 심심해요.'}, metadatas={'_id': '1f6da739', 'timestamp': 1722791803.777767, 'session_id': 'f56f808b'})
INFO:docstore:Added message: Message(message={'role': 'assistant', 'content': '그렇다면 새로운 취미를 찾아보는 건 어떨까요? 예를 들어 사이클이나 런닝같은 활동적인 것들은 어떤가요?'}, metadatas={'_id': '3a66273a', 'timestamp': 1722791806.1985

### 쿼리와 유사한 대화 k개 검색

In [20]:
queries = [
    "밖에 날씨가 어때?",
    "심심한데 뭐 하면 좋을까요?",
    "음악을 배우고 싶어요",
    "심심한데 재밌는 얘기 해줄래?",
    "요즘 취미로 소설을 쓰고 있는데 재미있는 이야기 소재가 있을까?"
    "잘 자!!",
    "요즘 너무 덥고 습해서 미치겠어요. 날씨가 왜 이런건지.."
]

for query in queries:
    distances, indices = retriever.search_similar(query)
    FaissIndex.print_results(query, distances, indices, docstore)


User query: 밖에 날씨가 어때?
1. Distance: 0.7859041264810275, Index: 2
Content: ### 명령어: 오늘 날씨가 어때요?
### 응답: 죄송하지만 저는 실시간 날씨 정보를 제공할 수 없습니다. 대신 일반적인 날씨 관련 조언을 드릴 수 있어요.
2. Distance: 0.9484258388209601, Index: 3
Content: ### 명령어: 그럼 여름철 더위 대비 팁 좀 알려주세요.
### 응답: 여름철 더위 대비 팁을 알려드리겠습니다. 1. 충분한 수분 섭취 2. 헐렁한 옷 입기 3. 그늘에서 쉬기 4. 선크림 바르기 5. 에어컨 적절히 사용하기

User query: 심심한데 뭐 하면 좋을까요?
1. Distance: 0.7267355469029055, Index: 1
Content: ### 명령어: 요즘 삶이 너무 무료하고 심심해요.
### 응답: 그렇다면 새로운 취미를 찾아보는 건 어떨까요? 예를 들어 사이클이나 런닝같은 활동적인 것들은 어떤가요?
2. Distance: 0.8059714115882073, Index: 4
Content: ### 명령어: 고마워요. 이제 다른 주제로 넘어갈게요. 취미 추천해주세요.
### 응답: 취미 추천 드리겠습니다. 1. 독서 2. 요리 3. 그림 그리기 4. 운동 (조깅, 요가 등) 5. 악기 연주 6. 정원 가꾸기 7. 사진 촬영 8. 외국어 공부. 이 중에서 관심 있는 것이 있나요?

User query: 음악을 배우고 싶어요
1. Distance: 0.5752006998718946, Index: 5
Content: ### 명령어: 음악에 관심이 있어요. 피아노를 배우고 싶은데 어떻게 시작하면 좋을까요?
### 응답: 피아노를 배우고 싶으시군요! 좋은 선택입니다. 시작하는 방법을 알려드리겠습니다. 1. 기초 악보 읽기 연습 2. 간단한 디지털 피아노나 키보드 구매 3. 온라인 무료 레슨 영상 활용 4. 근처 음악학원 등록 고려 5. 매일 조금씩이라도 연습하

### 시간 가중치 변경에 따른 결과 비교

In [24]:
test_query = "너무 심심해요. 뭐 재미있는 거 없나요??"

for time_weight in [0, 0.25, 0.5, 0.75, 1]:
    distances, indices = retriever.search_similar(test_query, k=3, time_weight=time_weight)
    print(f"\n시간 가중치: {time_weight}")
    FaissRetriever.print_results(test_query, distances, indices, docstore)


시간 가중치: 0

User query: 너무 심심해요. 뭐 재미있는 거 없나요??
1. Distance: 0.6610167026519775, Index: 1
Content: ### 명령어: 요즘 삶이 너무 무료하고 심심해요.
### 응답: 그렇다면 새로운 취미를 찾아보는 건 어떨까요? 예를 들어 사이클이나 런닝같은 활동적인 것들은 어떤가요?
2. Distance: 0.8920535445213318, Index: 4
Content: ### 명령어: 고마워요. 이제 다른 주제로 넘어갈게요. 취미 추천해주세요.
### 응답: 취미 추천 드리겠습니다. 1. 독서 2. 요리 3. 그림 그리기 4. 운동 (조깅, 요가 등) 5. 악기 연주 6. 정원 가꾸기 7. 사진 촬영 8. 외국어 공부. 이 중에서 관심 있는 것이 있나요?
3. Distance: 0.9587647914886475, Index: 7
Content: ### 명령어: 네, 재미있었어요. 이제 자러 갈게요. 잘자요!
### 응답: 네, 잘 들어주셔서 감사합니다. 좋은 꿈 꾸세요! 편안한 밤 되세요. 내일도 좋은 하루 되길 바랄게요. 안녕히 주무세요!

시간 가중치: 0.25

User query: 너무 심심해요. 뭐 재미있는 거 없나요??
1. Distance: 0.7100004885545506, Index: 1
Content: ### 명령어: 요즘 삶이 너무 무료하고 심심해요.
### 응답: 그렇다면 새로운 취미를 찾아보는 건 어떨까요? 예를 들어 사이클이나 런닝같은 활동적인 것들은 어떤가요?
2. Distance: 0.7190735936164856, Index: 7
Content: ### 명령어: 네, 재미있었어요. 이제 자러 갈게요. 잘자요!
### 응답: 네, 잘 들어주셔서 감사합니다. 좋은 꿈 꾸세요! 편안한 밤 되세요. 내일도 좋은 하루 되길 바랄게요. 안녕히 주무세요!
3. Distance: 0.779737328144927, Index: 4
Content: ### 명령어: 고마워요. 이제 다